稀疏化比例 epsilon = 0.1

RAW_SCORE 这个文件中 对于每个mu值（可以看作第零个坐标）

第一个坐标是样本编号

第二个坐标 0 代表稀疏化的网络 1 代表原网络

第三个坐标是方法 ['Euclidean NMI', 'Spherical NMI', 'Euclidean ECS', 'Spherical ECS']

RAW_QF 这个文件中，对于每个mu值（可以看作第零个坐标）

第一个坐标是样本编号

第二个坐标 0 代表稀疏化的网络 1 代表原网络

In [1]:
#隐藏警告
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Generate an LFR Network and Draw it
import numpy as np
import networkx as nx
from networkx.generators.community import LFR_benchmark_graph


n = 1000
tau1 = 2  # Power-law exponent for the degree distribution
tau2 = 1.1  # Power-law exponent for the community size distribution
#mu = 0.1  # Mixing parameter
avg_deg = 25  # Average Degree
max_deg = int(0.1 * n)  # Max Degree
min_commu = 60  # Min Community Size
max_commu = int(0.1 * n)  # Max Community Size

#for mu in np.arange(0.1, 0.11, 0.1):

In [3]:
from Network import *
import numpy as np
    
import networkx as nx
import scipy.sparse

def to_networkx(self):
    if isinstance(self.graph, scipy.sparse.csr.csr_matrix):
        return nx.from_scipy_sparse_matrix(self.graph)
    else:
        return nx.from_numpy_array(self.graph)

In [4]:
#%%time
# This function takes the orthogonal part of G_sparse eigenvectors.
# Hopefully, we will get a better community detection result from this treatment.
import networkx as nx
import numpy as np

def compute_orthogonal_components(G, V):
    """
    Compute the component of each column of V orthogonal to the degree sequence vector of a graph.

    Parameters:
    G (networkx.classes.graph.Graph): The input graph.
    V (numpy.ndarray): The input 2D array.

    Returns:
    numpy.ndarray: The component of each column of V orthogonal to the degree sequence vector of the graph.
    """
    # get measure and constant vector
    mu = np.array([d for n, d in G.degree()])
    u = np.ones(G.number_of_nodes())

    # 
    orthogonal_components = np.zeros_like(V)

    # do orthogonal for each column
    for i in range(V.shape[1]):
        v = V[:, i]
        orthogonal_component = v - ((v @ (u * mu)) / (u @ (u * mu))) * u
        orthogonal_components[:, i] = orthogonal_component  # 

    return orthogonal_components

In [5]:
# This is the function for Laplacian Eigenmap using Cupy. The presence of GPU is required.

import numpy as np
import networkx as nx
import cupy as cp

def lap_cupy(graph, dim):
    """
    Compute the Laplacian embedding of a graph using CuPy.

    Parameters:
    graph (networkx.classes.graph.Graph): The input graph.
    dim (int): The dimension of the embedding.

    Returns:
    numpy.ndarray: The Laplacian embedding of the graph.
    """
    # Check inputs
    assert isinstance(graph, nx.Graph), "Input graph must be a NetworkX graph."
    assert isinstance(dim, int) and dim > 0, "Input dim must be a positive integer."
    assert dim < graph.number_of_nodes(), "Input dim must be less than the number of nodes in the graph."

    # Convert the adjacency matrix of the graph to a CuPy array
    A = cp.asarray(nx.adjacency_matrix(graph, nodelist=graph.nodes(), weight='weight').toarray(), dtype=cp.float64)

    # Compute L1 normalization along axis 1 (rows)
    row_sums = cp.linalg.norm(A, ord=1, axis=1)
    A /= row_sums.reshape(-1, 1)

    # Compute the eigenvalues and eigenvectors of I_n - A
    I_n = cp.eye(graph.number_of_nodes())
    w, v = cp.linalg.eigh(I_n - A)

    # Sort the eigenvectors by the real part of the eigenvalues
    v = v[:, cp.argsort(w.real)]

    # Return the embedding
    return v[:, 1:(dim+1)].get().real  # Explicitly convert to NumPy array using .get()

In [6]:
### KMeans Clustering using Euclidean and Spherical metrics
### Using NMI and ECSim for comparison
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_distances
from sklearn.preprocessing import normalize
from sklearn.metrics import normalized_mutual_info_score
import clusim.sim as sim
from clusim.clustering import Clustering


def euclid_membership(K, points):
    euc_kmeans = KMeans(n_clusters=K, n_init=10)
    euc_kmeans.fit(points)

    evala_euclid_membership = euc_kmeans.labels_
    return evala_euclid_membership

def cosine_membership(K, points):
    normalized_points = normalize(points)
    cos_kmeans = KMeans(n_clusters=K, n_init=10)
    cos_kmeans.fit(normalized_points)

    evala_cosine_membership = cos_kmeans.labels_
    return evala_cosine_membership

def calculate_score(evala, intr_list, K):
# evala is the embedding vectors
# intr_list is the intrinsic community strucuture
# K is the number of clusters in Kmeans
    return_val = [] # 首先准备好返回值 

    intr_clus = Clustering({i: [intr_list[i]] for i in range(len(intr_list))})

    evala_euclid_membership = euclid_membership(K, evala)

    evala_cosine_membership = cosine_membership(K, evala)

    ## compare with intrinsic community structure using NMI
    return_val.append(normalized_mutual_info_score(evala_euclid_membership, intr_list, average_method='arithmetic'))
    return_val.append(normalized_mutual_info_score(evala_cosine_membership, intr_list, average_method='arithmetic'))
    
    
    evala_euclid_clustering = Clustering(elm2clu_dict={i: [evala_euclid_membership[i]] for i in range(len(evala_euclid_membership))})
    evala_cosine_clustering = Clustering(elm2clu_dict={i: [evala_cosine_membership[i]] for i in range(len(evala_cosine_membership))})
    
    ## compare with intrinsic community structure using ECSim
    evala_euclid_similarity = sim.element_sim(intr_clus, evala_euclid_clustering, alpha=0.9)
    evala_cosine_similarity = sim.element_sim(intr_clus, evala_cosine_clustering, alpha=0.9)
    return_val.append(evala_euclid_similarity)
    return_val.append(evala_cosine_similarity)
    
    return return_val

In [7]:
import numpy as np

step_total  = 10
step_size = 0.05
MU = np.arange(step_size, step_size*step_total+0.01, step_size)

SAMPLE = 50

stat = np.zeros((step_total, 3, 4))

K = 15

RAW_SCORE={}

In [8]:
for mu in MU:
    RAW_SCORE[mu] = np.zeros((SAMPLE, 3, 4))
    for i in range(SAMPLE):
        G = LFR_benchmark_graph(
        n, tau1, tau2, mu, average_degree=avg_deg, max_degree=max_deg, min_community=min_commu, max_community=max_commu,
        seed = 10
        )

        # Remove multi-edges and self-loops from G
        G = nx.Graph(G)
        selfloop_edges = list(nx.selfloop_edges(G))
        G.remove_edges_from(selfloop_edges)

        # LFR 图是有内在的社群结构的，每个节点的社群存储在其 community 属性中，是一个 set
        # 通过运行循环，按照内在的社群结构给每个节点一个标签 即为其 intrinsic_membership
        # 为了方便 intrinsic_membership 一开始是作为一个 dict 存储的，后来将其转化为一个 list
        intrinsic_communities = {frozenset(G.nodes[v]["community"]) for v in G}
        intrinsic_membership = {}
        for node in range(G.number_of_nodes()):
            for index, inner_set in enumerate(intrinsic_communities):
                if node in inner_set:
                    intrinsic_membership[node] = index
                    break
        intrinsic_membership = list(intrinsic_membership.values())

        # Get the edge list and edge weights and transform G from NetworkX to Network

        # Get edge list as a numpy array
        edge_list = list(G.edges())
        edge_list = np.array(edge_list)

        # Get edge weights as a numpy array
        edge_weights = nx.get_edge_attributes(G, 'weight')
        edge_weights = np.array(edge_weights)

        edge_weights = [edge_weights[edge] if edge in edge_weights else 1 for edge in edge_list]

        Gn = Network(edge_list, edge_weights)
        # Gn is the representation of G in Network format

        # Calculate effective resistance
        epsilon=0.1
        method='spl'
        Effective_R = Gn.effR(epsilon, method)

        # spectral sparse version of G in Network format
        q = 10000
        seed = 2024
        Gn_Sparse = Gn.spl(q, Effective_R, seed=seed)

        # G_sparse is the spectral sparse version of G in NetworkX format    
        G_sparse = to_networkx(Gn_Sparse)
        # check whether the sparse version is connected or not
        # nx.is_connected(G_sparse)

        score_sparse_orth = calculate_score(
            compute_orthogonal_components(G, lap_cupy(G_sparse,K)),intrinsic_membership, len(np.unique(intrinsic_membership)))
        RAW_SCORE[mu][i,0] = score_sparse_orth
        score_sparse = calculate_score(lap_cupy(G_sparse,K),intrinsic_membership, len(np.unique(intrinsic_membership)))
        RAW_SCORE[mu][i,1] = score_sparse
        score_original = calculate_score(lap_cupy(G,K),intrinsic_membership, len(np.unique(intrinsic_membership)))
        RAW_SCORE[mu][i,2] = score_original
    stat[int(mu/step_size)-1] = np.mean(RAW_SCORE[mu],axis=0)

KeyboardInterrupt: 

In [ ]:
import pickle

# 将 RAW_SCORE 存储到文件
with open('raw_score.pkl', 'wb') as file:
    pickle.dump(RAW_SCORE, file)

print("RAW_SCORE 已成功存储到 raw_score.pkl 文件中")

#import pickle

# 从文件加载 RAW_SCORE
#with open('raw_score.pkl', 'rb') as file:
#    loaded_raw_score = pickle.load(file)

#print("RAW_SCORE 已成功从 raw_score.pkl 文件中加载")
#print(loaded_raw_score)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x=4

# 绘制每列的折线图
plt.plot(stat[:, 0, x], label='Column 1', color='r')  # 红色曲线
plt.plot(stat[:, 1, x], label='Column 2', color='g')  # 绿色曲线
plt.plot(stat[:, 2, x], label='Column 3', color='b')  # 蓝色曲线

# 添加图例
plt.legend()

# 添加标签
plt.xlabel('Step')
plt.ylabel('Value')
plt.title('Line Graph for Each Column')

# 显示图形
plt.show()